# 0.0. Imports

In [1]:
import mlflow
from mlflow.tracking import MlflowClient

In [2]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment("prob_loan")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1708001633686, experiment_id='1', last_update_time=1708001633686, lifecycle_stage='active', name='prob_loan', tags={}>

In [3]:
client = mlflow.tracking.MlflowClient()

# 1.0. Transition model stage

In [4]:
client.transition_model_version_stage(
    name='modelo.joblib',
    version='1',
    stage='Production'

)


/tmp/ipykernel_16231/1695114636.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.10.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1708007137892, current_stage='Production', description='', last_updated_timestamp=1708018303510, name='modelo.joblib', run_id='c0438bf1074343dc93eba9daee49e315', run_link='', source='mlflow-artifacts:/1/c0438bf1074343dc93eba9daee49e315/artifacts/modelo.joblib', status='READY', status_message='', tags={}, user_id='', version='1'>

# 2.0. Models details

In [5]:
model_version_details = client.get_model_version(
    name='modelo.joblib',
    version='1'
)

In [6]:
model_version_details

<ModelVersion: aliases=[], creation_timestamp=1708007137892, current_stage='Production', description='', last_updated_timestamp=1708018303510, name='modelo.joblib', run_id='c0438bf1074343dc93eba9daee49e315', run_link='', source='mlflow-artifacts:/1/c0438bf1074343dc93eba9daee49e315/artifacts/modelo.joblib', status='READY', status_message='', tags={}, user_id='', version='1'>

# 3.0. Get the latest version

In [7]:
client.get_latest_versions('modelo.joblib',
                           stages=['Production'])

/tmp/ipykernel_16231/2788765008.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.10.0/model-registry.html#migrating-from-stages
  client.get_latest_versions('modelo.joblib',


[<ModelVersion: aliases=[], creation_timestamp=1708007137892, current_stage='Production', description='', last_updated_timestamp=1708018303510, name='modelo.joblib', run_id='c0438bf1074343dc93eba9daee49e315', run_link='', source='mlflow-artifacts:/1/c0438bf1074343dc93eba9daee49e315/artifacts/modelo.joblib', status='READY', status_message='', tags={}, user_id='', version='1'>]

# 4.0 get latest version by name

In [20]:
model_name = "modelo.joblib"
model = client.get_registered_model(model_name)
run_id = model.latest_versions[-1].run_id

model_uri = "runs:/{}".format(run_id)


# 5.0 Register name

In [21]:
mv = client.create_model_version(model_name, model_uri, run_id)

2024/02/15 14:45:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo.joblib, version 12


# 6.0 Register Alias

In [22]:
client.set_registered_model_alias(model_name, "test", mv.version)

# 7.0 Get Model Version By Alias

In [23]:
alias_mv = client.get_model_version_by_alias(model_name, "test")

In [24]:
alias_mv

<ModelVersion: aliases=['test'], creation_timestamp=1708019118911, current_stage='None', description='', last_updated_timestamp=1708019118911, name='modelo.joblib', run_id='35062587905241039900d575e6c4f193', run_link='', source='runs:/35062587905241039900d575e6c4f193', status='READY', status_message='', tags={}, user_id='', version='12'>

# 8.0 get model

In [28]:
import mlflow.pyfunc

model_name = "modelo.joblib"
alias = "modelo"

champion_version = mlflow.pyfunc.load_model(f"models:/{alias_mv.name}@{alias_mv.aliases[0]}")


In [29]:
champion_version

mlflow.pyfunc.loaded_model:
  artifact_path: lr_model
  flavor: mlflow.sklearn
  run_id: de8742d3045946059a6a95ccd46319e8

In [31]:
print(f"models:/{champion_version.name}@{champion_version.aliases[0]}")

models:/modelo.joblib@test
